In [1]:
ROLE_COLS=["DevType"]
TECH_COLS=['LanguageHaveWorkedWith',
       'DatabaseHaveWorkedWith',
       'PlatformHaveWorkedWith',
       'WebframeHaveWorkedWith',
       'MiscTechHaveWorkedWith',
       'ToolsTechHaveWorkedWith',
       'NEWCollabToolsHaveWorkedWith']

DATA_DIR=r'..\data\processed\2_cleaned_data.pkl'
MLFLOW_TRACKING_URI = '../models/mlruns/'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from pathlib import Path
import mlflow 
from mlflow import MlflowClient

In [3]:
df=pd.read_pickle(DATA_DIR)
df

DevType                                                \
      Data or business analyst Data scientist or machine learning specialist   
0                            0                                             0   
3                            0                                             0   
4                            0                                             0   
8                            0                                             0   
9                            0                                             1   
...                        ...                                           ...   
83434                        0                                             0   
83435                        0                                             0   
83436                        0                                             1   
83437                        0                                             0   
83438                        0                                             0   

                                                                      \
      Database administrator DevOps specialist Developer, QA or test   
0                          0                 0                     0   
3                          0                 0                     0   
4                          0                 0                     0   
8                          0                 0                     0   
9                          0                 0                     0   
...                      ...               ...                   ...   
83434                      0                 0                     0   
83435                      0                 0                     0   
83436                      0                 1                     0   
83437                      0                 0                     0   
83438                      0                 0                     0   

                                                                         \
      Developer, back-end Developer, desktop or enterprise applications   
0                       0                                             0   
3                       0                                             0   
4                       0                                             1   
8                       0                                             0   
9                       0                                             0   
...                   ...                                           ...   
83434                   1                                             0   
83435                   0                                             0   
83436                   0                                             0   
83437                   1                                             0   
83438                   1                                             0   

                                                                        \
      Developer, embedded applications or devices Developer, front-end   
0                                               0                    0   
3                                               0                    1   
4                                               0                    0   
8                                               0                    1   
9                                               0                    0   
...                                           ...                  ...   
83434                                           0                    0   
83435                                           0                    0   
83436                                           0                    0   
83437                                           0                    0   
83438                                           0                    1   

                             ... feature_clusters                       \
      Developer, full-stack  ...

sampling


In [4]:
print(df["DevType"].sum(axis=0))

Data or business analyst                          1699
Data scientist or machine learning specialist     2591
Database administrator                            1174
DevOps specialist                                 2874
Developer, QA or test                             1113
Developer, back-end                              16977
Developer, desktop or enterprise applications     4863
Developer, embedded applications or devices       2162
Developer, front-end                              8938
Developer, full-stack                            20620
Developer, game or graphics                        919
Developer, mobile                                 4766
Engineer, data                                    1945
Engineer, site reliability                         890
Scientist                                         1191
System administrator                              1978
dtype: int64


In [5]:
samples_per_class=1200
roles=df["DevType"].copy()

all_indices=[]
for col in roles.columns:
    # print(col)

    aa=roles[col].loc[roles[col]==1]
    if (roles[col].sum()>1200):
        aa=aa.sample(samples_per_class,replace=False,random_state=0)
    if(roles[col].sum()<1200):
        aa=aa.sample(samples_per_class,replace=True,random_state=0)
    # print(df["DevType"][col][aa.index])
    all_indices.append(list(aa.index))
    # print(aa.index.values)
all_indices=sum(all_indices,[])
new_df=df.loc[all_indices]
new_df

DevType                                                \
      Data or business analyst Data scientist or machine learning specialist   
33883                        1                                             0   
250                          1                                             1   
9101                         1                                             0   
63309                        1                                             1   
32625                        1                                             0   
...                        ...                                           ...   
24718                        0                                             0   
42768                        0                                             0   
10847                        0                                             0   
44020                        0                                             0   
81352                        0                                             0   

                                                                      \
      Database administrator DevOps specialist Developer, QA or test   
33883                      0                 0                     0   
250                        1                 0                     0   
9101                       0                 0                     0   
63309                      0                 0                     0   
32625                      0                 0                     0   
...                      ...               ...                   ...   
24718                      0                 0                     0   
42768                      0                 0                     0   
10847                      1                 0                     0   
44020                      0                 0                     0   
81352                      0                 1                     0   

                                                                         \
      Developer, back-end Developer, desktop or enterprise applications   
33883                   0                                             0   
250                     0                                             0   
9101                    0                                             0   
63309                   0                                             0   
32625                   1                                             0   
...                   ...                                           ...   
24718                   0                                             1   
42768                   0                                             0   
10847                   1                                             0   
44020                   0                                             1   
81352                   0                                             0   

                                                                        \
      Developer, embedded applications or devices Developer, front-end   
33883                                           0                    1   
250                                             0                    0   
9101                                            0                    0   
63309                                           0                    0   
32625                                           0                    0   
...                                           ...                  ...   
24718                                           0                    0   
42768                                           0                    0   
10847                                           0                    0   
44020                                           0                    0   
81352                                           0                    0   

                             ... feature_clusters                       \
      Developer, full-stack  ...

In [6]:
new_df["DevType"].sum(axis=0)

Data or business analyst                         1965
Data scientist or machine learning specialist    2313
Database administrator                           1767
DevOps specialist                                2513
Developer, QA or test                            1490
Developer, back-end                              5855
Developer, desktop or enterprise applications    2736
Developer, embedded applications or devices      1836
Developer, front-end                             2638
Developer, full-stack                            5730
Developer, game or graphics                      1448
Developer, mobile                                2134
Engineer, data                                   2020
Engineer, site reliability                       1603
Scientist                                        1652
System administrator                             2307
dtype: int64

In [7]:
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True) 

In [8]:
# from mlflow import MlflowClient
# import mlflow
# # Initialize client and experiment
# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# client = MlflowClient()
# mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
# exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

In [8]:
X_train,X_test,y_train,y_test=train_test_split(new_df.drop("DevType",axis=1),new_df["DevType"],test_size=.3,random_state=52)


C:\Users\Mohamed Mosaad\AppData\Local\Temp\ipykernel_29392\2036909935.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train,X_test,y_train,y_test=train_test_split(new_df.drop("DevType",axis=1),new_df["DevType"],test_size=.3,random_state=52)


In [9]:
print(X_train.shape)
print(y_train.shape)

(13440, 145)
(13440, 16)


In [10]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client=MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp=client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
if exp is None:
    print("Experiment not found ,creating new experiment")
    experiment_id=mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
else:
    experiment_id=exp.experiment_id    

Experiment not found ,creating new experiment


In [12]:
lr=make_pipeline(StandardScaler(),MultiOutputClassifier(LogisticRegression()))
lr.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

In [13]:
y_train_predicted=pd.DataFrame(lr.predict(X_train),columns=y_train.columns)
y_test_predicted=pd.DataFrame(lr.predict(X_test),columns=y_train.columns)
y_train_predicted

,Data or business analyst,Data scientist or machine learning specialist,Database administrator,DevOps specialist,"Developer, QA or test","Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, front-end","Developer, full-stack","Developer, game or graphics","Developer, mobile","Engineer, data","Engineer, site reliability",Scientist,System administrator
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13435,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13436,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
13437,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
13438,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [14]:
train_score={}
for col in y_train.columns:
    # print(precision_score(y_train[col],y_train_predicted[col]))
    train_score[col]=[(score(y_train[col],y_train_predicted[col])) for score in [precision_score,recall_score,f1_score,accuracy_score]]

train_score=pd.DataFrame(train_score,index=['precision_score','recall_score','f1_score','accuracy_score']).T
# print(train_score)
# train_score
mean_train_scores=train_score.mean()
print(mean_train_scores)
train_score

precision_score    0.650021
recall_score       0.312087
f1_score           0.403336
accuracy_score     0.892494
dtype: float64


,precision_score,recall_score,f1_score,accuracy_score
Data or business analyst,0.644599,0.264097,0.374684,0.908110
Data scientist or machine learning specialist,0.719867,0.534238,0.613314,0.918750
Database administrator,0.533333,0.057878,0.104423,0.908110
DevOps specialist,0.680782,0.350867,0.463072,0.891815
"Developer, QA or test",0.352941,0.005731,0.011278,0.921726
"Developer, back-end",0.618873,0.327754,0.428549,0.731920
"Developer, desktop or enterprise applications",0.655417,0.189913,0.294493,0.868452
"Developer, embedded applications or devices",0.682014,0.369735,0.479514,0.923438
"Developer, front-end",0.663992,0.367166,0.472857,0.890179
"Developer, full-stack",0.671990,0.550163,0.605005,0.787426


In [15]:
test_score={}
for col in y_test.columns:
    # print(precision_score(y_train[col],y_train_predicted[col]))
    test_score[col]=[(score(y_test[col],y_test_predicted[col])) for score in [precision_score,recall_score,f1_score,accuracy_score]]

test_score=pd.DataFrame(test_score,index=['precision_score','recall_score','f1_score','accuracy_score']).T

mean_test_scores=test_score.mean()
print(mean_test_scores)
test_score

precision_score    0.620225
recall_score       0.294902
f1_score           0.380612
accuracy_score     0.889095
dtype: float64


,precision_score,recall_score,f1_score,accuracy_score
Data or business analyst,0.582677,0.262411,0.361858,0.909375
Data scientist or machine learning specialist,0.710744,0.497110,0.585034,0.915278
Database administrator,0.509091,0.053537,0.096886,0.909375
DevOps specialist,0.650124,0.360882,0.464128,0.894965
"Developer, QA or test",0.545455,0.013544,0.026432,0.923264
"Developer, back-end",0.578616,0.318523,0.410867,0.725174
"Developer, desktop or enterprise applications",0.504098,0.155107,0.237223,0.862674
"Developer, embedded applications or devices",0.610759,0.348375,0.443678,0.915972
"Developer, front-end",0.653165,0.308982,0.419512,0.876042
"Developer, full-stack",0.660912,0.545921,0.597938,0.776563


LOG run

1_prepare

In [16]:
import pickle
data_details={'dara_path':DATA_DIR,
              'trainig_indices':X_train.index.tolist(),
              'testing_indices':X_test.index.tolist(),
              'feature_names':X_train.columns.droplevel(0).tolist(),
              'target_names':y_train.columns.tolist()
              }
with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [17]:
model = {"model_description": "Baseline model: Logistic Regression ",
         "model_details": str(lr),
         "model_object": lr} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [18]:
classes_metrics = {"train_scores": train_score, 
                   "test_scores":  test_score}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [19]:
# import mlflow
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # Log pickles 
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 